In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pickle

In [4]:
wisconsin_districts = pd.read_csv('data/Wisconsin/Wisconsin_with districts.csv')

In [5]:
wisconsin_districts

,Unnamed: 0,COUNTY,STUDENTS_TESTED,EXAM_COUNT,EXAMS_3_OR_ABOVE,DISTRICT_NAME,Year,PERCENT_3_OR_ABOVE,LATITUDE,LONGITUDE
0,0,Adams,45,76,22,Adams-Friendship Area,2018,28.947368,44.019695,-89.775534
1,1,Green,6,7,4,Albany,2018,57.142857,42.707785,-89.437063
2,2,Kewaunee,21,21,9,Algoma,2018,42.857143,44.608888,-87.432589
3,3,Portage,7,8,5,Almond-Bancroft,2018,62.500000,44.284011,-89.360630
4,4,Eau Claire,64,107,79,Altoona,2018,73.831776,44.804682,-91.443215
...,...,...,...,...,...,...,...,...,...,...
1485,1485,Dane,38,72,52,Wisconsin Heights,2022,72.222222,43.135655,-89.737996
1486,1486,Wood,117,155,105,Wisconsin Rapids,2022,67.741935,44.383576,-89.817346
1487,1487,Shawano,12,19,8,Wittenberg-Birnamwood,2022,42.105263,44.810379,-89.208677
1488,1488,Juneau,6,7,3,Wonewoc-Union Center,2022,42.857143,43.656537,-90.205711


In [6]:
wisconsin_districts18 = wisconsin_districts[wisconsin_districts['Year'] == 2018]
wisconsin_districts19 = wisconsin_districts[wisconsin_districts['Year'] == 2019]
wisconsin_districts20 = wisconsin_districts[wisconsin_districts['Year'] == 2020]
wisconsin_districts21 = wisconsin_districts[wisconsin_districts['Year'] == 2021]
wisconsin_districts22 = wisconsin_districts[wisconsin_districts['Year'] == 2022]

In [7]:
wisconsin_districts22

,Unnamed: 0,COUNTY,STUDENTS_TESTED,EXAM_COUNT,EXAMS_3_OR_ABOVE,DISTRICT_NAME,Year,PERCENT_3_OR_ABOVE,LATITUDE,LONGITUDE
1192,1192,Adams,42,79,22,Adams-Friendship Area,2022,27.848101,44.019695,-89.775534
1193,1193,Kewaunee,12,12,4,Algoma,2022,33.333333,44.608888,-87.432589
1194,1194,Eau Claire,68,97,77,Altoona,2022,79.381443,44.804682,-91.443215
1195,1195,Polk,64,76,35,Amery,2022,46.052632,45.306906,-92.362137
1196,1196,Langlade,52,68,30,Antigo Unified,2022,44.117647,45.140245,-89.152335
...,...,...,...,...,...,...,...,...,...,...
1485,1485,Dane,38,72,52,Wisconsin Heights,2022,72.222222,43.135655,-89.737996
1486,1486,Wood,117,155,105,Wisconsin Rapids,2022,67.741935,44.383576,-89.817346
1487,1487,Shawano,12,19,8,Wittenberg-Birnamwood,2022,42.105263,44.810379,-89.208677
1488,1488,Juneau,6,7,3,Wonewoc-Union Center,2022,42.857143,43.656537,-90.205711


In [20]:
fig = px.density_mapbox(wisconsin_districts22, lat='LATITUDE', lon='LONGITUDE', z='PERCENT_3_OR_ABOVE', radius=10,
                        center=dict(lat=44.8, lon=-89), zoom=5,
                        mapbox_style="open-street-map")
fig.update_layout(
    title='2022-23: AP Scores 3-5 by school districts of Wisconsin'
)

fig.show()
with open('data/WI_pickled/WI_AP_pass_rate_by_school_districts.pkl', 'wb') as f:
    pickle.dump(fig, f)

In [9]:
wisconsin = pd.read_csv('data/Wisconsin/train_test_split/Wisconsin_closest_five_method.csv')

In [10]:
wisconsin = wisconsin[wisconsin.columns[1:]]
wisconsin.columns

Index(['COUNTY', 'STUDENTS_TESTED', 'EXAM_COUNT', 'EXAMS_3_OR_ABOVE',
       'PERCENT_3_OR_ABOVE', 'Year', 'Latitude', 'Longitude', 'population',
       'per_capita_income', 'closest_five_r1r2_avg', 'closest_five_public_avg',
       'closest_five_private_nfp_avg', 'closest_five_landgrnt_avg',
       'closest_five_stem_avg', 'closest_five_avg_enrollment_r1r2',
       'closest_five_avg_enrollment_public',
       'closest_five_avg_enrollment_private_nfp',
       'closest_five_avg_enrollment_landgrnt',
       'closest_five_avg_enrollment_stem', 'closest_five_avg_dormrooms_r1r2',
       'closest_five_avg_dormrooms_public',
       'closest_five_avg_dormrooms_private_nfp',
       'closest_five_avg_dormrooms_landgrant',
       'closest_five_avg_dormrooms_stem'],
      dtype='object')

In [28]:
wisconsin22 = wisconsin[wisconsin['Year'] == 2022]

In [11]:
## Features indicated by shapley to be significant
significant_features = ['per_capita_income','population','closest_five_private_nfp_avg','closest_five_avg_dormrooms_private_nfp','closest_five_public_avg']

In [39]:

fig = go.Figure(data=go.Scatter(
    x=np.round(wisconsin22['per_capita_income'], decimals=-1),
    y=np.round(wisconsin22['PERCENT_3_OR_ABOVE'], decimals=1),
    mode='markers',
    marker=dict(
        size=7,
        color=wisconsin22['PERCENT_3_OR_ABOVE'],
        colorscale='Bluered',
        symbol='circle',
        showscale=True,
        colorbar=dict(title='% Score 3-5'),
    ),
    showlegend=False,
    text=wisconsin22['COUNTY'],
    name=''))
fig.show()

In [54]:
fig = px.scatter(data_frame=wisconsin22,x='per_capita_income', y='PERCENT_3_OR_ABOVE',trendline='lowess',trendline_color_override='darkblue',width=800, height=600,color='PERCENT_3_OR_ABOVE',
                 title='Pass rate vs Per Capita Income 2022-23')
fig.show()
with open('data/WI_pickled/WI_AP_pass_rate_by_counties_vs_income.pkl', 'wb') as f:
    pickle.dump(fig, f)

In [55]:
fig = px.scatter(data_frame=wisconsin22,x='population', y='PERCENT_3_OR_ABOVE',trendline='ols',trendline_color_override='darkblue',width=800, height=600,color='PERCENT_3_OR_ABOVE',title='Pass rate vs Population')
fig.show()
with open('data/WI_pickled/WI_AP_pass_rate_by_counties_vs_population.pkl', 'wb') as f:
    pickle.dump(fig, f)

In [56]:
fig = px.scatter(data_frame=wisconsin22,x='closest_five_public_avg', y='PERCENT_3_OR_ABOVE',trendline='ols',trendline_color_override='darkblue',width=800, height=600,
                 color='PERCENT_3_OR_ABOVE',title='Pass rate vs Avg. distance to closest 5 public universities')
fig.show()
with open('data/WI_pickled/WI_AP_pass_rate_by_counties_vs_avg._dist_public.pkl', 'wb') as f:
    pickle.dump(fig, f)

In [57]:
fig = px.scatter(data_frame=wisconsin22,x='closest_five_private_nfp_avg', y='PERCENT_3_OR_ABOVE',trendline='ols',trendline_color_override='darkblue',width=800, height=600,
                 color='PERCENT_3_OR_ABOVE',title='Pass rate vs Avg. distance to closest five private universities')
fig.show()
with open('data/WI_pickled/WI_AP_pass_rate_by_counties_vs_avg_five_private.pkl', 'wb') as f:
    pickle.dump(fig, f)

In [58]:
fig = px.scatter(data_frame=wisconsin22,x='closest_five_avg_dormrooms_private_nfp', y='PERCENT_3_OR_ABOVE',trendline='ols',trendline_color_override='darkblue',width=800, height=600,
                 color='PERCENT_3_OR_ABOVE',title='Pass rate vs Avg. no. of dormrooms in the closest five private universities')
fig.show()
with open('data/WI_pickled/WI_AP_pass_rate_by_counties_vs_avg_five_private_dormrooms.pkl', 'wb') as f:
    pickle.dump(fig, f)